In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn import cross_validation

c:\users\fiona\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('train_clean.csv')

In [3]:
# create samples that always contain all queries of a srch_id
srch_id_groups = train.groupby('srch_id')
all_srch_ids = train.srch_id.unique()
num_samples = int(0.05*len(all_srch_ids))
train_sample_ids = np.random.choice(all_srch_ids, num_samples, replace=False)
remaining_ids = list(set(all_srch_ids)-set(train_sample_ids))
test_sample_ids = np.random.choice(remaining_ids, num_samples, replace=False)

train_sample = pd.concat([srch_id_groups.get_group(group) for group in train_sample_ids])
test_sample = pd.concat([srch_id_groups.get_group(group) for group in test_sample_ids])

In [7]:
def fit_model(model, training_set, target, features):
    return model.fit(training_set[features], target)
    
def predict(model, test_set, features, num_categories=2):
    probs = model.predict_proba(test_set[features])
    test_set['probability1'] = probs[:,-1]
    test_set['probability2'] = probs[:,-2]
    ranking_list = []
    srch_id_list = []
    num_ids = len(test_set.srch_id.unique())
    current = 1
    for i in test_set.srch_id.unique():
        ranking = test_set[test_set.srch_id==i]
        srch_id_list.extend(ranking.srch_id.values)
        ranking = ranking.as_matrix(columns=['srch_id','prop_id','probability1','probability2'])
        ranking = ranking[ranking[:,2].argsort(kind='mergesort')]
        if num_categories > 2:
            ranking = ranking[ranking[:,3].argsort(kind='mergesort')]
        ranking_list = np.concatenate((ranking_list,np.flipud(ranking[:,1])))
        print('{} out of {} srch_ids done'.format(current, num_ids))
        current += 1
    test_set.drop('probability1', axis=1, inplace=True)
    test_set.drop('probability2', axis=1, inplace=True)
    return pd.DataFrame(np.array([srch_id_list, ranking_list]).T, columns=['srch_id','prop_id'])

In [5]:
train.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,star_diff_bool,price_diff,price_diff_bool,visitor_hist_starrating_filled,visitor_hist_adr_usd_filled,willingness_to_pay,booked_clicked_combined,prop_desirability,star_diff_filled,price_diff_filled
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,0,NaN,0,3.472818,202.072673,0,0,0.016340,0.472818,97.302673
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,0,NaN,0,3.472818,202.072673,0,0,0.015437,0.527182,31.332673
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,0,NaN,0,3.472818,202.072673,0,0,0.003630,0.472818,22.272673
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,0,NaN,0,3.472818,202.072673,0,0,0.013043,1.472818,400.697327
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,0,NaN,0,3.472818,202.072673,0,0,0.036090,0.527182,58.492673


In [5]:
features = ['site_id', 'visitor_location_country_id', 'prop_country_id', 'prop_review_score', 'prop_brand_bool', 'prop_location_score1', 'prop_location_score2', 'promotion_flag', 'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool', 'srch_query_affinity_score', 'orig_destination_distance', 'random_bool', 'star_diff_bool', 'price_diff_bool', 'star_diff_filled', 'price_diff_filled', 'willingness_to_pay', 'prop_desirability']

In [7]:
rf_model = fit_model(RandomForestClassifier(), train_sample, train_sample.booking_bool, features)

In [8]:
rf_prediction = predict(rf_model, test_sample, features)

1 out of 19979 srch_ids done
2 out of 19979 srch_ids done
3 out of 19979 srch_ids done
4 out of 19979 srch_ids done
5 out of 19979 srch_ids done
6 out of 19979 srch_ids done
7 out of 19979 srch_ids done
8 out of 19979 srch_ids done
9 out of 19979 srch_ids done
10 out of 19979 srch_ids done
11 out of 19979 srch_ids done
12 out of 19979 srch_ids done
13 out of 19979 srch_ids done
14 out of 19979 srch_ids done
15 out of 19979 srch_ids done
16 out of 19979 srch_ids done
17 out of 19979 srch_ids done
18 out of 19979 srch_ids done
19 out of 19979 srch_ids done
20 out of 19979 srch_ids done
21 out of 19979 srch_ids done
22 out of 19979 srch_ids done
23 out of 19979 srch_ids done
24 out of 19979 srch_ids done
25 out of 19979 srch_ids done
26 out of 19979 srch_ids done
27 out of 19979 srch_ids done
28 out of 19979 srch_ids done
29 out of 19979 srch_ids done
30 out of 19979 srch_ids done
31 out of 19979 srch_ids done
32 out of 19979 srch_ids done
33 out of 19979 srch_ids done
34 out of 19979 src

304 out of 19979 srch_ids done
305 out of 19979 srch_ids done
306 out of 19979 srch_ids done
307 out of 19979 srch_ids done
308 out of 19979 srch_ids done
309 out of 19979 srch_ids done
310 out of 19979 srch_ids done
311 out of 19979 srch_ids done
312 out of 19979 srch_ids done
313 out of 19979 srch_ids done
314 out of 19979 srch_ids done
315 out of 19979 srch_ids done
316 out of 19979 srch_ids done
317 out of 19979 srch_ids done
318 out of 19979 srch_ids done
319 out of 19979 srch_ids done
320 out of 19979 srch_ids done
321 out of 19979 srch_ids done
322 out of 19979 srch_ids done
323 out of 19979 srch_ids done
324 out of 19979 srch_ids done
325 out of 19979 srch_ids done
326 out of 19979 srch_ids done
327 out of 19979 srch_ids done
328 out of 19979 srch_ids done
329 out of 19979 srch_ids done
330 out of 19979 srch_ids done
331 out of 19979 srch_ids done
332 out of 19979 srch_ids done
333 out of 19979 srch_ids done
334 out of 19979 srch_ids done
335 out of 19979 srch_ids done
336 out 

603 out of 19979 srch_ids done
604 out of 19979 srch_ids done
605 out of 19979 srch_ids done
606 out of 19979 srch_ids done
607 out of 19979 srch_ids done
608 out of 19979 srch_ids done
609 out of 19979 srch_ids done
610 out of 19979 srch_ids done
611 out of 19979 srch_ids done
612 out of 19979 srch_ids done
613 out of 19979 srch_ids done
614 out of 19979 srch_ids done
615 out of 19979 srch_ids done
616 out of 19979 srch_ids done
617 out of 19979 srch_ids done
618 out of 19979 srch_ids done
619 out of 19979 srch_ids done
620 out of 19979 srch_ids done
621 out of 19979 srch_ids done
622 out of 19979 srch_ids done
623 out of 19979 srch_ids done
624 out of 19979 srch_ids done
625 out of 19979 srch_ids done
626 out of 19979 srch_ids done
627 out of 19979 srch_ids done
628 out of 19979 srch_ids done
629 out of 19979 srch_ids done
630 out of 19979 srch_ids done
631 out of 19979 srch_ids done
632 out of 19979 srch_ids done
633 out of 19979 srch_ids done
634 out of 19979 srch_ids done
635 out 

909 out of 19979 srch_ids done
910 out of 19979 srch_ids done
911 out of 19979 srch_ids done
912 out of 19979 srch_ids done
913 out of 19979 srch_ids done
914 out of 19979 srch_ids done
915 out of 19979 srch_ids done
916 out of 19979 srch_ids done
917 out of 19979 srch_ids done
918 out of 19979 srch_ids done
919 out of 19979 srch_ids done
920 out of 19979 srch_ids done
921 out of 19979 srch_ids done
922 out of 19979 srch_ids done
923 out of 19979 srch_ids done
924 out of 19979 srch_ids done
925 out of 19979 srch_ids done
926 out of 19979 srch_ids done
927 out of 19979 srch_ids done
928 out of 19979 srch_ids done
929 out of 19979 srch_ids done
930 out of 19979 srch_ids done
931 out of 19979 srch_ids done
932 out of 19979 srch_ids done
933 out of 19979 srch_ids done
934 out of 19979 srch_ids done
935 out of 19979 srch_ids done
936 out of 19979 srch_ids done
937 out of 19979 srch_ids done
938 out of 19979 srch_ids done
939 out of 19979 srch_ids done
940 out of 19979 srch_ids done
941 out 

1209 out of 19979 srch_ids done
1210 out of 19979 srch_ids done
1211 out of 19979 srch_ids done
1212 out of 19979 srch_ids done
1213 out of 19979 srch_ids done
1214 out of 19979 srch_ids done
1215 out of 19979 srch_ids done
1216 out of 19979 srch_ids done
1217 out of 19979 srch_ids done
1218 out of 19979 srch_ids done
1219 out of 19979 srch_ids done
1220 out of 19979 srch_ids done
1221 out of 19979 srch_ids done
1222 out of 19979 srch_ids done
1223 out of 19979 srch_ids done
1224 out of 19979 srch_ids done
1225 out of 19979 srch_ids done
1226 out of 19979 srch_ids done
1227 out of 19979 srch_ids done
1228 out of 19979 srch_ids done
1229 out of 19979 srch_ids done
1230 out of 19979 srch_ids done
1231 out of 19979 srch_ids done
1232 out of 19979 srch_ids done
1233 out of 19979 srch_ids done
1234 out of 19979 srch_ids done
1235 out of 19979 srch_ids done
1236 out of 19979 srch_ids done
1237 out of 19979 srch_ids done
1238 out of 19979 srch_ids done
1239 out of 19979 srch_ids done
1240 out

...

18942 out of 19979 srch_ids done
18943 out of 19979 srch_ids done
18944 out of 19979 srch_ids done
18945 out of 19979 srch_ids done
18946 out of 19979 srch_ids done
18947 out of 19979 srch_ids done
18948 out of 19979 srch_ids done
18949 out of 19979 srch_ids done
18950 out of 19979 srch_ids done
18951 out of 19979 srch_ids done
18952 out of 19979 srch_ids done
18953 out of 19979 srch_ids done
18954 out of 19979 srch_ids done
18955 out of 19979 srch_ids done
18956 out of 19979 srch_ids done
18957 out of 19979 srch_ids done
18958 out of 19979 srch_ids done
18959 out of 19979 srch_ids done
18960 out of 19979 srch_ids done
18961 out of 19979 srch_ids done
18962 out of 19979 srch_ids done
18963 out of 19979 srch_ids done
18964 out of 19979 srch_ids done
18965 out of 19979 srch_ids done
18966 out of 19979 srch_ids done
18967 out of 19979 srch_ids done
18968 out of 19979 srch_ids done
18969 out of 19979 srch_ids done
18970 out of 19979 srch_ids done
18971 out of 19979 srch_ids done
18972 out 

19205 out of 19979 srch_ids done
19206 out of 19979 srch_ids done
19207 out of 19979 srch_ids done
19208 out of 19979 srch_ids done
19209 out of 19979 srch_ids done
19210 out of 19979 srch_ids done
19211 out of 19979 srch_ids done
19212 out of 19979 srch_ids done
19213 out of 19979 srch_ids done
19214 out of 19979 srch_ids done
19215 out of 19979 srch_ids done
19216 out of 19979 srch_ids done
19217 out of 19979 srch_ids done
19218 out of 19979 srch_ids done
19219 out of 19979 srch_ids done
19220 out of 19979 srch_ids done
19221 out of 19979 srch_ids done
19222 out of 19979 srch_ids done
19223 out of 19979 srch_ids done
19224 out of 19979 srch_ids done
19225 out of 19979 srch_ids done
19226 out of 19979 srch_ids done
19227 out of 19979 srch_ids done
19228 out of 19979 srch_ids done
19229 out of 19979 srch_ids done
19230 out of 19979 srch_ids done
19231 out of 19979 srch_ids done
19232 out of 19979 srch_ids done
19233 out of 19979 srch_ids done
19234 out of 19979 srch_ids done
19235 out 

19455 out of 19979 srch_ids done
19456 out of 19979 srch_ids done
19457 out of 19979 srch_ids done
19458 out of 19979 srch_ids done
19459 out of 19979 srch_ids done
19460 out of 19979 srch_ids done
19461 out of 19979 srch_ids done
19462 out of 19979 srch_ids done
19463 out of 19979 srch_ids done
19464 out of 19979 srch_ids done
19465 out of 19979 srch_ids done
19466 out of 19979 srch_ids done
19467 out of 19979 srch_ids done
19468 out of 19979 srch_ids done
19469 out of 19979 srch_ids done
19470 out of 19979 srch_ids done
19471 out of 19979 srch_ids done
19472 out of 19979 srch_ids done
19473 out of 19979 srch_ids done
19474 out of 19979 srch_ids done
19475 out of 19979 srch_ids done
19476 out of 19979 srch_ids done
19477 out of 19979 srch_ids done
19478 out of 19979 srch_ids done
19479 out of 19979 srch_ids done
19480 out of 19979 srch_ids done
19481 out of 19979 srch_ids done
19482 out of 19979 srch_ids done
19483 out of 19979 srch_ids done
19484 out of 19979 srch_ids done
19485 out 

19718 out of 19979 srch_ids done
19719 out of 19979 srch_ids done
19720 out of 19979 srch_ids done
19721 out of 19979 srch_ids done
19722 out of 19979 srch_ids done
19723 out of 19979 srch_ids done
19724 out of 19979 srch_ids done
19725 out of 19979 srch_ids done
19726 out of 19979 srch_ids done
19727 out of 19979 srch_ids done
19728 out of 19979 srch_ids done
19729 out of 19979 srch_ids done
19730 out of 19979 srch_ids done
19731 out of 19979 srch_ids done
19732 out of 19979 srch_ids done
19733 out of 19979 srch_ids done
19734 out of 19979 srch_ids done
19735 out of 19979 srch_ids done
19736 out of 19979 srch_ids done
19737 out of 19979 srch_ids done
19738 out of 19979 srch_ids done
19739 out of 19979 srch_ids done
19740 out of 19979 srch_ids done
19741 out of 19979 srch_ids done
19742 out of 19979 srch_ids done
19743 out of 19979 srch_ids done
19744 out of 19979 srch_ids done
19745 out of 19979 srch_ids done
19746 out of 19979 srch_ids done
19747 out of 19979 srch_ids done
19748 out 

19973 out of 19979 srch_ids done
19974 out of 19979 srch_ids done
19975 out of 19979 srch_ids done
19976 out of 19979 srch_ids done
19977 out of 19979 srch_ids done
19978 out of 19979 srch_ids done
19979 out of 19979 srch_ids done


In [10]:
rf_model.feature_importances_

array([0.61118779, 0.02433704, 0.01023348, 0.01006874, 0.34417295])

In [11]:
gb_model = fit_model(GradientBoostingClassifier(), train_sample, train_sample.booking_bool, features)

In [12]:
gb_prediction = predict(gb_model, test_sample, features)

1 out of 19979 srch_ids done
2 out of 19979 srch_ids done
3 out of 19979 srch_ids done
4 out of 19979 srch_ids done
5 out of 19979 srch_ids done
6 out of 19979 srch_ids done
7 out of 19979 srch_ids done
8 out of 19979 srch_ids done
9 out of 19979 srch_ids done
10 out of 19979 srch_ids done
11 out of 19979 srch_ids done
12 out of 19979 srch_ids done
13 out of 19979 srch_ids done
14 out of 19979 srch_ids done
15 out of 19979 srch_ids done
16 out of 19979 srch_ids done
17 out of 19979 srch_ids done
18 out of 19979 srch_ids done
19 out of 19979 srch_ids done
20 out of 19979 srch_ids done
21 out of 19979 srch_ids done
22 out of 19979 srch_ids done
23 out of 19979 srch_ids done
24 out of 19979 srch_ids done
25 out of 19979 srch_ids done
26 out of 19979 srch_ids done
27 out of 19979 srch_ids done
28 out of 19979 srch_ids done
29 out of 19979 srch_ids done
30 out of 19979 srch_ids done
31 out of 19979 srch_ids done
32 out of 19979 srch_ids done
33 out of 19979 srch_ids done
34 out of 19979 src

278 out of 19979 srch_ids done
279 out of 19979 srch_ids done
280 out of 19979 srch_ids done
281 out of 19979 srch_ids done
282 out of 19979 srch_ids done
283 out of 19979 srch_ids done
284 out of 19979 srch_ids done
285 out of 19979 srch_ids done
286 out of 19979 srch_ids done
287 out of 19979 srch_ids done
288 out of 19979 srch_ids done
289 out of 19979 srch_ids done
290 out of 19979 srch_ids done
291 out of 19979 srch_ids done
292 out of 19979 srch_ids done
293 out of 19979 srch_ids done
294 out of 19979 srch_ids done
295 out of 19979 srch_ids done
296 out of 19979 srch_ids done
297 out of 19979 srch_ids done
298 out of 19979 srch_ids done
299 out of 19979 srch_ids done
300 out of 19979 srch_ids done
301 out of 19979 srch_ids done
302 out of 19979 srch_ids done
303 out of 19979 srch_ids done
304 out of 19979 srch_ids done
305 out of 19979 srch_ids done
306 out of 19979 srch_ids done
307 out of 19979 srch_ids done
308 out of 19979 srch_ids done
309 out of 19979 srch_ids done
310 out 

568 out of 19979 srch_ids done
569 out of 19979 srch_ids done
570 out of 19979 srch_ids done
571 out of 19979 srch_ids done
572 out of 19979 srch_ids done
573 out of 19979 srch_ids done
574 out of 19979 srch_ids done
575 out of 19979 srch_ids done
576 out of 19979 srch_ids done
577 out of 19979 srch_ids done
578 out of 19979 srch_ids done
579 out of 19979 srch_ids done
580 out of 19979 srch_ids done
581 out of 19979 srch_ids done
582 out of 19979 srch_ids done
583 out of 19979 srch_ids done
584 out of 19979 srch_ids done
585 out of 19979 srch_ids done
586 out of 19979 srch_ids done
587 out of 19979 srch_ids done
588 out of 19979 srch_ids done
589 out of 19979 srch_ids done
590 out of 19979 srch_ids done
591 out of 19979 srch_ids done
592 out of 19979 srch_ids done
593 out of 19979 srch_ids done
594 out of 19979 srch_ids done
595 out of 19979 srch_ids done
596 out of 19979 srch_ids done
597 out of 19979 srch_ids done
598 out of 19979 srch_ids done
599 out of 19979 srch_ids done
600 out 

856 out of 19979 srch_ids done
857 out of 19979 srch_ids done
858 out of 19979 srch_ids done
859 out of 19979 srch_ids done
860 out of 19979 srch_ids done
861 out of 19979 srch_ids done
862 out of 19979 srch_ids done
863 out of 19979 srch_ids done
864 out of 19979 srch_ids done
865 out of 19979 srch_ids done
866 out of 19979 srch_ids done
867 out of 19979 srch_ids done
868 out of 19979 srch_ids done
869 out of 19979 srch_ids done
870 out of 19979 srch_ids done
871 out of 19979 srch_ids done
872 out of 19979 srch_ids done
873 out of 19979 srch_ids done
874 out of 19979 srch_ids done
875 out of 19979 srch_ids done
876 out of 19979 srch_ids done
877 out of 19979 srch_ids done
878 out of 19979 srch_ids done
879 out of 19979 srch_ids done
880 out of 19979 srch_ids done
881 out of 19979 srch_ids done
882 out of 19979 srch_ids done
883 out of 19979 srch_ids done
884 out of 19979 srch_ids done
885 out of 19979 srch_ids done
886 out of 19979 srch_ids done
887 out of 19979 srch_ids done
888 out 

1131 out of 19979 srch_ids done
1132 out of 19979 srch_ids done
1133 out of 19979 srch_ids done
1134 out of 19979 srch_ids done
1135 out of 19979 srch_ids done
1136 out of 19979 srch_ids done
1137 out of 19979 srch_ids done
1138 out of 19979 srch_ids done
1139 out of 19979 srch_ids done
1140 out of 19979 srch_ids done
1141 out of 19979 srch_ids done
1142 out of 19979 srch_ids done
1143 out of 19979 srch_ids done
1144 out of 19979 srch_ids done
1145 out of 19979 srch_ids done
1146 out of 19979 srch_ids done
1147 out of 19979 srch_ids done
1148 out of 19979 srch_ids done
1149 out of 19979 srch_ids done
1150 out of 19979 srch_ids done
1151 out of 19979 srch_ids done
1152 out of 19979 srch_ids done
1153 out of 19979 srch_ids done
1154 out of 19979 srch_ids done
1155 out of 19979 srch_ids done
1156 out of 19979 srch_ids done
1157 out of 19979 srch_ids done
1158 out of 19979 srch_ids done
1159 out of 19979 srch_ids done
1160 out of 19979 srch_ids done
1161 out of 19979 srch_ids done
1162 out

...

18787 out of 19979 srch_ids done
18788 out of 19979 srch_ids done
18789 out of 19979 srch_ids done
18790 out of 19979 srch_ids done
18791 out of 19979 srch_ids done
18792 out of 19979 srch_ids done
18793 out of 19979 srch_ids done
18794 out of 19979 srch_ids done
18795 out of 19979 srch_ids done
18796 out of 19979 srch_ids done
18797 out of 19979 srch_ids done
18798 out of 19979 srch_ids done
18799 out of 19979 srch_ids done
18800 out of 19979 srch_ids done
18801 out of 19979 srch_ids done
18802 out of 19979 srch_ids done
18803 out of 19979 srch_ids done
18804 out of 19979 srch_ids done
18805 out of 19979 srch_ids done
18806 out of 19979 srch_ids done
18807 out of 19979 srch_ids done
18808 out of 19979 srch_ids done
18809 out of 19979 srch_ids done
18810 out of 19979 srch_ids done
18811 out of 19979 srch_ids done
18812 out of 19979 srch_ids done
18813 out of 19979 srch_ids done
18814 out of 19979 srch_ids done
18815 out of 19979 srch_ids done
18816 out of 19979 srch_ids done
18817 out 

19040 out of 19979 srch_ids done
19041 out of 19979 srch_ids done
19042 out of 19979 srch_ids done
19043 out of 19979 srch_ids done
19044 out of 19979 srch_ids done
19045 out of 19979 srch_ids done
19046 out of 19979 srch_ids done
19047 out of 19979 srch_ids done
19048 out of 19979 srch_ids done
19049 out of 19979 srch_ids done
19050 out of 19979 srch_ids done
19051 out of 19979 srch_ids done
19052 out of 19979 srch_ids done
19053 out of 19979 srch_ids done
19054 out of 19979 srch_ids done
19055 out of 19979 srch_ids done
19056 out of 19979 srch_ids done
19057 out of 19979 srch_ids done
19058 out of 19979 srch_ids done
19059 out of 19979 srch_ids done
19060 out of 19979 srch_ids done
19061 out of 19979 srch_ids done
19062 out of 19979 srch_ids done
19063 out of 19979 srch_ids done
19064 out of 19979 srch_ids done
19065 out of 19979 srch_ids done
19066 out of 19979 srch_ids done
19067 out of 19979 srch_ids done
19068 out of 19979 srch_ids done
19069 out of 19979 srch_ids done
19070 out 

19290 out of 19979 srch_ids done
19291 out of 19979 srch_ids done
19292 out of 19979 srch_ids done
19293 out of 19979 srch_ids done
19294 out of 19979 srch_ids done
19295 out of 19979 srch_ids done
19296 out of 19979 srch_ids done
19297 out of 19979 srch_ids done
19298 out of 19979 srch_ids done
19299 out of 19979 srch_ids done
19300 out of 19979 srch_ids done
19301 out of 19979 srch_ids done
19302 out of 19979 srch_ids done
19303 out of 19979 srch_ids done
19304 out of 19979 srch_ids done
19305 out of 19979 srch_ids done
19306 out of 19979 srch_ids done
19307 out of 19979 srch_ids done
19308 out of 19979 srch_ids done
19309 out of 19979 srch_ids done
19310 out of 19979 srch_ids done
19311 out of 19979 srch_ids done
19312 out of 19979 srch_ids done
19313 out of 19979 srch_ids done
19314 out of 19979 srch_ids done
19315 out of 19979 srch_ids done
19316 out of 19979 srch_ids done
19317 out of 19979 srch_ids done
19318 out of 19979 srch_ids done
19319 out of 19979 srch_ids done
19320 out 

19540 out of 19979 srch_ids done
19541 out of 19979 srch_ids done
19542 out of 19979 srch_ids done
19543 out of 19979 srch_ids done
19544 out of 19979 srch_ids done
19545 out of 19979 srch_ids done
19546 out of 19979 srch_ids done
19547 out of 19979 srch_ids done
19548 out of 19979 srch_ids done
19549 out of 19979 srch_ids done
19550 out of 19979 srch_ids done
19551 out of 19979 srch_ids done
19552 out of 19979 srch_ids done
19553 out of 19979 srch_ids done
19554 out of 19979 srch_ids done
19555 out of 19979 srch_ids done
19556 out of 19979 srch_ids done
19557 out of 19979 srch_ids done
19558 out of 19979 srch_ids done
19559 out of 19979 srch_ids done
19560 out of 19979 srch_ids done
19561 out of 19979 srch_ids done
19562 out of 19979 srch_ids done
19563 out of 19979 srch_ids done
19564 out of 19979 srch_ids done
19565 out of 19979 srch_ids done
19566 out of 19979 srch_ids done
19567 out of 19979 srch_ids done
19568 out of 19979 srch_ids done
19569 out of 19979 srch_ids done
19570 out 

19812 out of 19979 srch_ids done
19813 out of 19979 srch_ids done
19814 out of 19979 srch_ids done
19815 out of 19979 srch_ids done
19816 out of 19979 srch_ids done
19817 out of 19979 srch_ids done
19818 out of 19979 srch_ids done
19819 out of 19979 srch_ids done
19820 out of 19979 srch_ids done
19821 out of 19979 srch_ids done
19822 out of 19979 srch_ids done
19823 out of 19979 srch_ids done
19824 out of 19979 srch_ids done
19825 out of 19979 srch_ids done
19826 out of 19979 srch_ids done
19827 out of 19979 srch_ids done
19828 out of 19979 srch_ids done
19829 out of 19979 srch_ids done
19830 out of 19979 srch_ids done
19831 out of 19979 srch_ids done
19832 out of 19979 srch_ids done
19833 out of 19979 srch_ids done
19834 out of 19979 srch_ids done
19835 out of 19979 srch_ids done
19836 out of 19979 srch_ids done
19837 out of 19979 srch_ids done
19838 out of 19979 srch_ids done
19839 out of 19979 srch_ids done
19840 out of 19979 srch_ids done
19841 out of 19979 srch_ids done
19842 out 

In [13]:
gb_model.feature_importances_

array([0.46120413, 0.16406037, 0.03786363, 0.13283631, 0.20403556])

In [6]:
def ideal_ranking(ground_truth):
    ideal_ranking = ground_truth.sort_values(by=['booking_bool', 'click_bool'], ascending=False)
    return ideal_ranking.reset_index(drop=True)
    
def DCG(prediction, ground_truth):
    dcg = []
    for row in prediction.itertuples():
        prop_id = getattr(row, 'prop_id')
        rank = getattr(row, 'Index') + 1
        booked = ground_truth.set_index('prop_id').loc[prop_id,'booking_bool']
        clicked = ground_truth.set_index('prop_id').loc[prop_id,'click_bool']
        score = 5 if booked else 1 if clicked else 0
        dcg.append((2**score - 1)/(np.log2(rank + 1)))
    return sum(dcg)

def nDCG(prediction, ground_truth):
    prediction.reset_index(drop=True, inplace=True)
    dcg = DCG(prediction, ground_truth)
    idcg = DCG(ideal_ranking(ground_truth), ground_truth)
    return(dcg/idcg)

def overall_nDCG(prediction, ground_truth):
    ids = prediction.srch_id.unique()
    srch_id_groups_pred = prediction.groupby('srch_id')
    srch_id_groups_gt = ground_truth.groupby('srch_id')
    all_nDCG = [nDCG(srch_id_groups_pred.get_group(i), srch_id_groups_gt.get_group(i)) for i in ids]
    #print(all_nDCG)
    #print(np.mean(all_nDCG))
    return np.mean(all_nDCG)

In [15]:
overall_nDCG(rf_prediction, test_sample)

[0.27023815442731974, 0.27894294565112987, 0.21810429198553155, 0.2890648263178879, 1.0, 0.2890648263178879, 0.27894294565112987, 0.227670248696953, 0.43067655807339306, 0.2559580248098155, 0.33333333333333337, 0.3562071871080222, 0.4306765580733931, 0.6309297535714575, 0.232926559138352, 0.33333333333333337, 0.31546487678572877, 0.26264953503719357, 0.6309297535714575, 0.3126166209409659, 0.27023815442731974, 0.3010299956639812, 0.3333333333333333, 0.3333333333333333, 0.38685280723454163, 0.23137821315975915, 0.22106472945750372, 0.21274605355336315, 0.22106472945750374, 0.24465054211822604, 0.24465054211822604, 0.27023815442731974, 0.6309297535714575, 0.31546487678572877, 0.3576691957798507, 1.0, 0.2890648263178879, 1.0, 0.3010299956639812, 0.27023815442731974, 0.3562071871080222, 0.38685280723454163, 0.23981246656813146, 0.23981246656813146, 0.21810429198553155, 0.6309297535714575, 0.3333333333333333, 0.4306765580733931, 0.6309297535714575, 0.24465054211822604, 0.27023815442731974, 

In [16]:
overall_nDCG(gb_prediction, test_sample)

[0.38685280723454163, 0.6309297535714575, 0.3010299956639812, 0.33333333333333337, 0.36271774012986924, 1.0, 0.2559580248098155, 0.23540891336663824, 0.5, 0.24465054211822604, 0.24465054211822604, 0.27023815442731974, 0.3562071871080222, 0.4306765580733931, 0.6264368622704714, 0.33333333333333337, 0.27894294565112987, 0.2890648263178879, 0.33333333333333337, 0.26179073784279006, 0.25, 0.24465054211822604, 0.2890648263178879, 0.2890648263178879, 0.22767024869695296, 0.27023815442731974, 0.3010299956639812, 0.22424382421757544, 0.43067655807339306, 0.27894294565112987, 0.22424382421757544, 0.23137821315975918, 0.5, 0.2890648263178879, 0.3264445986455623, 0.24465054211822604, 0.23137821315975915, 0.2890648263178879, 0.2037950470905062, 0.27023815442731974, 0.6309297535714575, 0.31546487678572877, 0.3010299956639812, 0.33333333333333337, 0.1965616322328226, 0.27023815442731974, 0.3562071871080222, 0.6309297535714575, 0.25, 0.23540891336663824, 0.27023815442731974, 0.38685280723454163, 0.20

In [5]:
def fit_predict_evaluate(data, model, features, target_column, num_categories=2, cv_fold=10):
    labels = np.mod(data.srch_id.values, cv_fold) # arrange srch_ids in k groups for k-fold cross-validation
    cv_idx = cross_validation.LeaveOneLabelOut(labels)
    #print(cv_idx)
    scores = []
    feature_importances = []
    for train_idx, test_idx in cv_idx:
        #print("%s %s" % (train_idx, test_idx))
        train, test = data.iloc[train_idx], data.iloc[test_idx]
        print('fit model')
        model = fit_model(model, train, train[target_column], features)
        feature_importances.append(model.feature_importances_)
        print('predict')
        prediction = predict(model, test, features, num_categories)
        print('evaluate')
        scores.append(overall_nDCG(prediction, test))
    return scores, feature_importances

In [11]:
scores, feature_importances = fit_predict_evaluate(train_sample, RandomForestClassifier(), features, 'booking_bool')

fit model
predict


c:\users\fiona\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


1 out of 1003 srch_ids done
2 out of 1003 srch_ids done
3 out of 1003 srch_ids done
4 out of 1003 srch_ids done
5 out of 1003 srch_ids done
6 out of 1003 srch_ids done
7 out of 1003 srch_ids done
8 out of 1003 srch_ids done
9 out of 1003 srch_ids done
10 out of 1003 srch_ids done
11 out of 1003 srch_ids done
12 out of 1003 srch_ids done
13 out of 1003 srch_ids done
14 out of 1003 srch_ids done
15 out of 1003 srch_ids done
16 out of 1003 srch_ids done
17 out of 1003 srch_ids done
18 out of 1003 srch_ids done
19 out of 1003 srch_ids done
20 out of 1003 srch_ids done
21 out of 1003 srch_ids done
22 out of 1003 srch_ids done
23 out of 1003 srch_ids done
24 out of 1003 srch_ids done
25 out of 1003 srch_ids done
26 out of 1003 srch_ids done
27 out of 1003 srch_ids done
28 out of 1003 srch_ids done
29 out of 1003 srch_ids done
30 out of 1003 srch_ids done
31 out of 1003 srch_ids done
32 out of 1003 srch_ids done
33 out of 1003 srch_ids done
34 out of 1003 srch_ids done
35 out of 1003 srch_ids

280 out of 1003 srch_ids done
281 out of 1003 srch_ids done
282 out of 1003 srch_ids done
283 out of 1003 srch_ids done
284 out of 1003 srch_ids done
285 out of 1003 srch_ids done
286 out of 1003 srch_ids done
287 out of 1003 srch_ids done
288 out of 1003 srch_ids done
289 out of 1003 srch_ids done
290 out of 1003 srch_ids done
291 out of 1003 srch_ids done
292 out of 1003 srch_ids done
293 out of 1003 srch_ids done
294 out of 1003 srch_ids done
295 out of 1003 srch_ids done
296 out of 1003 srch_ids done
297 out of 1003 srch_ids done
298 out of 1003 srch_ids done
299 out of 1003 srch_ids done
300 out of 1003 srch_ids done
301 out of 1003 srch_ids done
302 out of 1003 srch_ids done
303 out of 1003 srch_ids done
304 out of 1003 srch_ids done
305 out of 1003 srch_ids done
306 out of 1003 srch_ids done
307 out of 1003 srch_ids done
308 out of 1003 srch_ids done
309 out of 1003 srch_ids done
310 out of 1003 srch_ids done
311 out of 1003 srch_ids done
312 out of 1003 srch_ids done
313 out of

572 out of 1003 srch_ids done
573 out of 1003 srch_ids done
574 out of 1003 srch_ids done
575 out of 1003 srch_ids done
576 out of 1003 srch_ids done
577 out of 1003 srch_ids done
578 out of 1003 srch_ids done
579 out of 1003 srch_ids done
580 out of 1003 srch_ids done
581 out of 1003 srch_ids done
582 out of 1003 srch_ids done
583 out of 1003 srch_ids done
584 out of 1003 srch_ids done
585 out of 1003 srch_ids done
586 out of 1003 srch_ids done
587 out of 1003 srch_ids done
588 out of 1003 srch_ids done
589 out of 1003 srch_ids done
590 out of 1003 srch_ids done
591 out of 1003 srch_ids done
592 out of 1003 srch_ids done
593 out of 1003 srch_ids done
594 out of 1003 srch_ids done
595 out of 1003 srch_ids done
596 out of 1003 srch_ids done
597 out of 1003 srch_ids done
598 out of 1003 srch_ids done
599 out of 1003 srch_ids done
600 out of 1003 srch_ids done
601 out of 1003 srch_ids done
602 out of 1003 srch_ids done
603 out of 1003 srch_ids done
604 out of 1003 srch_ids done
605 out of

...

1 out of 1004 srch_ids done
2 out of 1004 srch_ids done
3 out of 1004 srch_ids done
4 out of 1004 srch_ids done
5 out of 1004 srch_ids done
6 out of 1004 srch_ids done
7 out of 1004 srch_ids done
8 out of 1004 srch_ids done
9 out of 1004 srch_ids done
10 out of 1004 srch_ids done
11 out of 1004 srch_ids done
12 out of 1004 srch_ids done
13 out of 1004 srch_ids done
14 out of 1004 srch_ids done
15 out of 1004 srch_ids done
16 out of 1004 srch_ids done
17 out of 1004 srch_ids done
18 out of 1004 srch_ids done
19 out of 1004 srch_ids done
20 out of 1004 srch_ids done
21 out of 1004 srch_ids done
22 out of 1004 srch_ids done
23 out of 1004 srch_ids done
24 out of 1004 srch_ids done
25 out of 1004 srch_ids done
26 out of 1004 srch_ids done
27 out of 1004 srch_ids done
28 out of 1004 srch_ids done
29 out of 1004 srch_ids done
30 out of 1004 srch_ids done
31 out of 1004 srch_ids done
32 out of 1004 srch_ids done
33 out of 1004 srch_ids done
34 out of 1004 srch_ids done
35 out of 1004 srch_ids

308 out of 1004 srch_ids done
309 out of 1004 srch_ids done
310 out of 1004 srch_ids done
311 out of 1004 srch_ids done
312 out of 1004 srch_ids done
313 out of 1004 srch_ids done
314 out of 1004 srch_ids done
315 out of 1004 srch_ids done
316 out of 1004 srch_ids done
317 out of 1004 srch_ids done
318 out of 1004 srch_ids done
319 out of 1004 srch_ids done
320 out of 1004 srch_ids done
321 out of 1004 srch_ids done
322 out of 1004 srch_ids done
323 out of 1004 srch_ids done
324 out of 1004 srch_ids done
325 out of 1004 srch_ids done
326 out of 1004 srch_ids done
327 out of 1004 srch_ids done
328 out of 1004 srch_ids done
329 out of 1004 srch_ids done
330 out of 1004 srch_ids done
331 out of 1004 srch_ids done
332 out of 1004 srch_ids done
333 out of 1004 srch_ids done
334 out of 1004 srch_ids done
335 out of 1004 srch_ids done
336 out of 1004 srch_ids done
337 out of 1004 srch_ids done
338 out of 1004 srch_ids done
339 out of 1004 srch_ids done
340 out of 1004 srch_ids done
341 out of

609 out of 1004 srch_ids done
610 out of 1004 srch_ids done
611 out of 1004 srch_ids done
612 out of 1004 srch_ids done
613 out of 1004 srch_ids done
614 out of 1004 srch_ids done
615 out of 1004 srch_ids done
616 out of 1004 srch_ids done
617 out of 1004 srch_ids done
618 out of 1004 srch_ids done
619 out of 1004 srch_ids done
620 out of 1004 srch_ids done
621 out of 1004 srch_ids done
622 out of 1004 srch_ids done
623 out of 1004 srch_ids done
624 out of 1004 srch_ids done
625 out of 1004 srch_ids done
626 out of 1004 srch_ids done
627 out of 1004 srch_ids done
628 out of 1004 srch_ids done
629 out of 1004 srch_ids done
630 out of 1004 srch_ids done
631 out of 1004 srch_ids done
632 out of 1004 srch_ids done
633 out of 1004 srch_ids done
634 out of 1004 srch_ids done
635 out of 1004 srch_ids done
636 out of 1004 srch_ids done
637 out of 1004 srch_ids done
638 out of 1004 srch_ids done
639 out of 1004 srch_ids done
640 out of 1004 srch_ids done
641 out of 1004 srch_ids done
642 out of

899 out of 1004 srch_ids done
900 out of 1004 srch_ids done
901 out of 1004 srch_ids done
902 out of 1004 srch_ids done
903 out of 1004 srch_ids done
904 out of 1004 srch_ids done
905 out of 1004 srch_ids done
906 out of 1004 srch_ids done
907 out of 1004 srch_ids done
908 out of 1004 srch_ids done
909 out of 1004 srch_ids done
910 out of 1004 srch_ids done
911 out of 1004 srch_ids done
912 out of 1004 srch_ids done
913 out of 1004 srch_ids done
914 out of 1004 srch_ids done
915 out of 1004 srch_ids done
916 out of 1004 srch_ids done
917 out of 1004 srch_ids done
918 out of 1004 srch_ids done
919 out of 1004 srch_ids done
920 out of 1004 srch_ids done
921 out of 1004 srch_ids done
922 out of 1004 srch_ids done
923 out of 1004 srch_ids done
924 out of 1004 srch_ids done
925 out of 1004 srch_ids done
926 out of 1004 srch_ids done
927 out of 1004 srch_ids done
928 out of 1004 srch_ids done
929 out of 1004 srch_ids done
930 out of 1004 srch_ids done
931 out of 1004 srch_ids done
932 out of

c:\users\fiona\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
scores

[0.42660953826271597,
 0.41892584774575964,
 0.422525080862133,
 0.420029965342799,
 0.4207587296432993,
 0.4269241852472829,
 0.4269507864027895,
 0.41261083589224856,
 0.4272233761978273,
 0.42910857294225146]

In [25]:
mean_fi = np.mean(feature_importances, axis=0)

In [24]:
pd.DataFrame({'feature':features, 'importance':mean_fi})

,feature,importance
0,site_id,0.021476
1,visitor_location_country_id,0.026543
2,prop_country_id,0.029656
3,prop_review_score,0.038057
4,prop_brand_bool,0.014832
5,prop_location_score1,0.087377
6,prop_location_score2,0.120382
7,promotion_flag,0.006976
8,srch_length_of_stay,0.032974
9,srch_booking_window,0.081726


In [26]:
gb_scores, gb_feature_importances = fit_predict_evaluate(train_sample, GradientBoostingClassifier(), features, 'booking_bool')

fit model
predict


c:\users\fiona\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


1 out of 1003 srch_ids done
2 out of 1003 srch_ids done
3 out of 1003 srch_ids done
4 out of 1003 srch_ids done
5 out of 1003 srch_ids done
6 out of 1003 srch_ids done
7 out of 1003 srch_ids done
8 out of 1003 srch_ids done
9 out of 1003 srch_ids done
10 out of 1003 srch_ids done
11 out of 1003 srch_ids done
12 out of 1003 srch_ids done
13 out of 1003 srch_ids done
14 out of 1003 srch_ids done
15 out of 1003 srch_ids done
16 out of 1003 srch_ids done
17 out of 1003 srch_ids done
18 out of 1003 srch_ids done
19 out of 1003 srch_ids done
20 out of 1003 srch_ids done
21 out of 1003 srch_ids done
22 out of 1003 srch_ids done
23 out of 1003 srch_ids done
24 out of 1003 srch_ids done
25 out of 1003 srch_ids done
26 out of 1003 srch_ids done
27 out of 1003 srch_ids done
28 out of 1003 srch_ids done
29 out of 1003 srch_ids done
30 out of 1003 srch_ids done
31 out of 1003 srch_ids done
32 out of 1003 srch_ids done
33 out of 1003 srch_ids done
34 out of 1003 srch_ids done
35 out of 1003 srch_ids

334 out of 1003 srch_ids done
335 out of 1003 srch_ids done
336 out of 1003 srch_ids done
337 out of 1003 srch_ids done
338 out of 1003 srch_ids done
339 out of 1003 srch_ids done
340 out of 1003 srch_ids done
341 out of 1003 srch_ids done
342 out of 1003 srch_ids done
343 out of 1003 srch_ids done
344 out of 1003 srch_ids done
345 out of 1003 srch_ids done
346 out of 1003 srch_ids done
347 out of 1003 srch_ids done
348 out of 1003 srch_ids done
349 out of 1003 srch_ids done
350 out of 1003 srch_ids done
351 out of 1003 srch_ids done
352 out of 1003 srch_ids done
353 out of 1003 srch_ids done
354 out of 1003 srch_ids done
355 out of 1003 srch_ids done
356 out of 1003 srch_ids done
357 out of 1003 srch_ids done
358 out of 1003 srch_ids done
359 out of 1003 srch_ids done
360 out of 1003 srch_ids done
361 out of 1003 srch_ids done
362 out of 1003 srch_ids done
363 out of 1003 srch_ids done
364 out of 1003 srch_ids done
365 out of 1003 srch_ids done
366 out of 1003 srch_ids done
367 out of

630 out of 1003 srch_ids done
631 out of 1003 srch_ids done
632 out of 1003 srch_ids done
633 out of 1003 srch_ids done
634 out of 1003 srch_ids done
635 out of 1003 srch_ids done
636 out of 1003 srch_ids done
637 out of 1003 srch_ids done
638 out of 1003 srch_ids done
639 out of 1003 srch_ids done
640 out of 1003 srch_ids done
641 out of 1003 srch_ids done
642 out of 1003 srch_ids done
643 out of 1003 srch_ids done
644 out of 1003 srch_ids done
645 out of 1003 srch_ids done
646 out of 1003 srch_ids done
647 out of 1003 srch_ids done
648 out of 1003 srch_ids done
649 out of 1003 srch_ids done
650 out of 1003 srch_ids done
651 out of 1003 srch_ids done
652 out of 1003 srch_ids done
653 out of 1003 srch_ids done
654 out of 1003 srch_ids done
655 out of 1003 srch_ids done
656 out of 1003 srch_ids done
657 out of 1003 srch_ids done
658 out of 1003 srch_ids done
659 out of 1003 srch_ids done
660 out of 1003 srch_ids done
661 out of 1003 srch_ids done
662 out of 1003 srch_ids done
663 out of

...

697 out of 1019 srch_ids done
698 out of 1019 srch_ids done
699 out of 1019 srch_ids done
700 out of 1019 srch_ids done
701 out of 1019 srch_ids done
702 out of 1019 srch_ids done
703 out of 1019 srch_ids done
704 out of 1019 srch_ids done
705 out of 1019 srch_ids done
706 out of 1019 srch_ids done
707 out of 1019 srch_ids done
708 out of 1019 srch_ids done
709 out of 1019 srch_ids done
710 out of 1019 srch_ids done
711 out of 1019 srch_ids done
712 out of 1019 srch_ids done
713 out of 1019 srch_ids done
714 out of 1019 srch_ids done
715 out of 1019 srch_ids done
716 out of 1019 srch_ids done
717 out of 1019 srch_ids done
718 out of 1019 srch_ids done
719 out of 1019 srch_ids done
720 out of 1019 srch_ids done
721 out of 1019 srch_ids done
722 out of 1019 srch_ids done
723 out of 1019 srch_ids done
724 out of 1019 srch_ids done
725 out of 1019 srch_ids done
726 out of 1019 srch_ids done
727 out of 1019 srch_ids done
728 out of 1019 srch_ids done
729 out of 1019 srch_ids done
730 out of

988 out of 1019 srch_ids done
989 out of 1019 srch_ids done
990 out of 1019 srch_ids done
991 out of 1019 srch_ids done
992 out of 1019 srch_ids done
993 out of 1019 srch_ids done
994 out of 1019 srch_ids done
995 out of 1019 srch_ids done
996 out of 1019 srch_ids done
997 out of 1019 srch_ids done
998 out of 1019 srch_ids done
999 out of 1019 srch_ids done
1000 out of 1019 srch_ids done
1001 out of 1019 srch_ids done
1002 out of 1019 srch_ids done
1003 out of 1019 srch_ids done
1004 out of 1019 srch_ids done
1005 out of 1019 srch_ids done
1006 out of 1019 srch_ids done
1007 out of 1019 srch_ids done
1008 out of 1019 srch_ids done
1009 out of 1019 srch_ids done
1010 out of 1019 srch_ids done
1011 out of 1019 srch_ids done
1012 out of 1019 srch_ids done
1013 out of 1019 srch_ids done
1014 out of 1019 srch_ids done
1015 out of 1019 srch_ids done
1016 out of 1019 srch_ids done
1017 out of 1019 srch_ids done
1018 out of 1019 srch_ids done
1019 out of 1019 srch_ids done
evaluate
fit model
p

309 out of 1004 srch_ids done
310 out of 1004 srch_ids done
311 out of 1004 srch_ids done
312 out of 1004 srch_ids done
313 out of 1004 srch_ids done
314 out of 1004 srch_ids done
315 out of 1004 srch_ids done
316 out of 1004 srch_ids done
317 out of 1004 srch_ids done
318 out of 1004 srch_ids done
319 out of 1004 srch_ids done
320 out of 1004 srch_ids done
321 out of 1004 srch_ids done
322 out of 1004 srch_ids done
323 out of 1004 srch_ids done
324 out of 1004 srch_ids done
325 out of 1004 srch_ids done
326 out of 1004 srch_ids done
327 out of 1004 srch_ids done
328 out of 1004 srch_ids done
329 out of 1004 srch_ids done
330 out of 1004 srch_ids done
331 out of 1004 srch_ids done
332 out of 1004 srch_ids done
333 out of 1004 srch_ids done
334 out of 1004 srch_ids done
335 out of 1004 srch_ids done
336 out of 1004 srch_ids done
337 out of 1004 srch_ids done
338 out of 1004 srch_ids done
339 out of 1004 srch_ids done
340 out of 1004 srch_ids done
341 out of 1004 srch_ids done
342 out of

605 out of 1004 srch_ids done
606 out of 1004 srch_ids done
607 out of 1004 srch_ids done
608 out of 1004 srch_ids done
609 out of 1004 srch_ids done
610 out of 1004 srch_ids done
611 out of 1004 srch_ids done
612 out of 1004 srch_ids done
613 out of 1004 srch_ids done
614 out of 1004 srch_ids done
615 out of 1004 srch_ids done
616 out of 1004 srch_ids done
617 out of 1004 srch_ids done
618 out of 1004 srch_ids done
619 out of 1004 srch_ids done
620 out of 1004 srch_ids done
621 out of 1004 srch_ids done
622 out of 1004 srch_ids done
623 out of 1004 srch_ids done
624 out of 1004 srch_ids done
625 out of 1004 srch_ids done
626 out of 1004 srch_ids done
627 out of 1004 srch_ids done
628 out of 1004 srch_ids done
629 out of 1004 srch_ids done
630 out of 1004 srch_ids done
631 out of 1004 srch_ids done
632 out of 1004 srch_ids done
633 out of 1004 srch_ids done
634 out of 1004 srch_ids done
635 out of 1004 srch_ids done
636 out of 1004 srch_ids done
637 out of 1004 srch_ids done
638 out of

899 out of 1004 srch_ids done
900 out of 1004 srch_ids done
901 out of 1004 srch_ids done
902 out of 1004 srch_ids done
903 out of 1004 srch_ids done
904 out of 1004 srch_ids done
905 out of 1004 srch_ids done
906 out of 1004 srch_ids done
907 out of 1004 srch_ids done
908 out of 1004 srch_ids done
909 out of 1004 srch_ids done
910 out of 1004 srch_ids done
911 out of 1004 srch_ids done
912 out of 1004 srch_ids done
913 out of 1004 srch_ids done
914 out of 1004 srch_ids done
915 out of 1004 srch_ids done
916 out of 1004 srch_ids done
917 out of 1004 srch_ids done
918 out of 1004 srch_ids done
919 out of 1004 srch_ids done
920 out of 1004 srch_ids done
921 out of 1004 srch_ids done
922 out of 1004 srch_ids done
923 out of 1004 srch_ids done
924 out of 1004 srch_ids done
925 out of 1004 srch_ids done
926 out of 1004 srch_ids done
927 out of 1004 srch_ids done
928 out of 1004 srch_ids done
929 out of 1004 srch_ids done
930 out of 1004 srch_ids done
931 out of 1004 srch_ids done
932 out of

In [27]:
gb_scores

[0.4900557217826057,
 0.48965004048889216,
 0.5084453674834735,
 0.48332831566670503,
 0.489154837253973,
 0.4858982735860839,
 0.5045564395168768,
 0.493550856659835,
 0.49865369768541423,
 0.5089056533403141]

In [28]:
gb_mean_fi = np.mean(gb_feature_importances, axis=0)

In [30]:
pd.DataFrame({'feature':features, 'importance':gb_mean_fi})

,feature,importance
0,site_id,0.009375
1,visitor_location_country_id,0.007271
2,prop_country_id,0.020727
3,prop_review_score,0.009946
4,prop_brand_bool,0.002149
5,prop_location_score1,0.038463
6,prop_location_score2,0.126341
7,promotion_flag,0.022108
8,srch_length_of_stay,0.012228
9,srch_booking_window,0.024594


In [8]:
gb_scores_3cat, gb_feature_importances_3cat = fit_predict_evaluate(train_sample, GradientBoostingClassifier(), features, 'booked_clicked_combined', num_categories=3)

fit model
predict


c:\users\fiona\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\fiona\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


1 out of 1065 srch_ids done
2 out of 1065 srch_ids done
3 out of 1065 srch_ids done
4 out of 1065 srch_ids done
5 out of 1065 srch_ids done
6 out of 1065 srch_ids done
7 out of 1065 srch_ids done
8 out of 1065 srch_ids done
9 out of 1065 srch_ids done
10 out of 1065 srch_ids done
11 out of 1065 srch_ids done
12 out of 1065 srch_ids done
13 out of 1065 srch_ids done
14 out of 1065 srch_ids done
15 out of 1065 srch_ids done
16 out of 1065 srch_ids done
17 out of 1065 srch_ids done
18 out of 1065 srch_ids done
19 out of 1065 srch_ids done
20 out of 1065 srch_ids done
21 out of 1065 srch_ids done
22 out of 1065 srch_ids done
23 out of 1065 srch_ids done
24 out of 1065 srch_ids done
25 out of 1065 srch_ids done
26 out of 1065 srch_ids done
27 out of 1065 srch_ids done
28 out of 1065 srch_ids done
29 out of 1065 srch_ids done
30 out of 1065 srch_ids done
31 out of 1065 srch_ids done
32 out of 1065 srch_ids done
33 out of 1065 srch_ids done
34 out of 1065 srch_ids done
35 out of 1065 srch_ids

294 out of 1065 srch_ids done
295 out of 1065 srch_ids done
296 out of 1065 srch_ids done
297 out of 1065 srch_ids done
298 out of 1065 srch_ids done
299 out of 1065 srch_ids done
300 out of 1065 srch_ids done
301 out of 1065 srch_ids done
302 out of 1065 srch_ids done
303 out of 1065 srch_ids done
304 out of 1065 srch_ids done
305 out of 1065 srch_ids done
306 out of 1065 srch_ids done
307 out of 1065 srch_ids done
308 out of 1065 srch_ids done
309 out of 1065 srch_ids done
310 out of 1065 srch_ids done
311 out of 1065 srch_ids done
312 out of 1065 srch_ids done
313 out of 1065 srch_ids done
314 out of 1065 srch_ids done
315 out of 1065 srch_ids done
316 out of 1065 srch_ids done
317 out of 1065 srch_ids done
318 out of 1065 srch_ids done
319 out of 1065 srch_ids done
320 out of 1065 srch_ids done
321 out of 1065 srch_ids done
322 out of 1065 srch_ids done
323 out of 1065 srch_ids done
324 out of 1065 srch_ids done
325 out of 1065 srch_ids done
326 out of 1065 srch_ids done
327 out of

608 out of 1065 srch_ids done
609 out of 1065 srch_ids done
610 out of 1065 srch_ids done
611 out of 1065 srch_ids done
612 out of 1065 srch_ids done
613 out of 1065 srch_ids done
614 out of 1065 srch_ids done
615 out of 1065 srch_ids done
616 out of 1065 srch_ids done
617 out of 1065 srch_ids done
618 out of 1065 srch_ids done
619 out of 1065 srch_ids done
620 out of 1065 srch_ids done
621 out of 1065 srch_ids done
622 out of 1065 srch_ids done
623 out of 1065 srch_ids done
624 out of 1065 srch_ids done
625 out of 1065 srch_ids done
626 out of 1065 srch_ids done
627 out of 1065 srch_ids done
628 out of 1065 srch_ids done
629 out of 1065 srch_ids done
630 out of 1065 srch_ids done
631 out of 1065 srch_ids done
632 out of 1065 srch_ids done
633 out of 1065 srch_ids done
634 out of 1065 srch_ids done
635 out of 1065 srch_ids done
636 out of 1065 srch_ids done
637 out of 1065 srch_ids done
638 out of 1065 srch_ids done
639 out of 1065 srch_ids done
640 out of 1065 srch_ids done
641 out of

...

282 out of 962 srch_ids done
283 out of 962 srch_ids done
284 out of 962 srch_ids done
285 out of 962 srch_ids done
286 out of 962 srch_ids done
287 out of 962 srch_ids done
288 out of 962 srch_ids done
289 out of 962 srch_ids done
290 out of 962 srch_ids done
291 out of 962 srch_ids done
292 out of 962 srch_ids done
293 out of 962 srch_ids done
294 out of 962 srch_ids done
295 out of 962 srch_ids done
296 out of 962 srch_ids done
297 out of 962 srch_ids done
298 out of 962 srch_ids done
299 out of 962 srch_ids done
300 out of 962 srch_ids done
301 out of 962 srch_ids done
302 out of 962 srch_ids done
303 out of 962 srch_ids done
304 out of 962 srch_ids done
305 out of 962 srch_ids done
306 out of 962 srch_ids done
307 out of 962 srch_ids done
308 out of 962 srch_ids done
309 out of 962 srch_ids done
310 out of 962 srch_ids done
311 out of 962 srch_ids done
312 out of 962 srch_ids done
313 out of 962 srch_ids done
314 out of 962 srch_ids done
315 out of 962 srch_ids done
316 out of 962

657 out of 962 srch_ids done
658 out of 962 srch_ids done
659 out of 962 srch_ids done
660 out of 962 srch_ids done
661 out of 962 srch_ids done
662 out of 962 srch_ids done
663 out of 962 srch_ids done
664 out of 962 srch_ids done
665 out of 962 srch_ids done
666 out of 962 srch_ids done
667 out of 962 srch_ids done
668 out of 962 srch_ids done
669 out of 962 srch_ids done
670 out of 962 srch_ids done
671 out of 962 srch_ids done
672 out of 962 srch_ids done
673 out of 962 srch_ids done
674 out of 962 srch_ids done
675 out of 962 srch_ids done
676 out of 962 srch_ids done
677 out of 962 srch_ids done
678 out of 962 srch_ids done
679 out of 962 srch_ids done
680 out of 962 srch_ids done
681 out of 962 srch_ids done
682 out of 962 srch_ids done
683 out of 962 srch_ids done
684 out of 962 srch_ids done
685 out of 962 srch_ids done
686 out of 962 srch_ids done
687 out of 962 srch_ids done
688 out of 962 srch_ids done
689 out of 962 srch_ids done
690 out of 962 srch_ids done
691 out of 962

fit model
predict
1 out of 989 srch_ids done
2 out of 989 srch_ids done
3 out of 989 srch_ids done
4 out of 989 srch_ids done
5 out of 989 srch_ids done
6 out of 989 srch_ids done
7 out of 989 srch_ids done
8 out of 989 srch_ids done
9 out of 989 srch_ids done
10 out of 989 srch_ids done
11 out of 989 srch_ids done
12 out of 989 srch_ids done
13 out of 989 srch_ids done
14 out of 989 srch_ids done
15 out of 989 srch_ids done
16 out of 989 srch_ids done
17 out of 989 srch_ids done
18 out of 989 srch_ids done
19 out of 989 srch_ids done
20 out of 989 srch_ids done
21 out of 989 srch_ids done
22 out of 989 srch_ids done
23 out of 989 srch_ids done
24 out of 989 srch_ids done
25 out of 989 srch_ids done
26 out of 989 srch_ids done
27 out of 989 srch_ids done
28 out of 989 srch_ids done
29 out of 989 srch_ids done
30 out of 989 srch_ids done
31 out of 989 srch_ids done
32 out of 989 srch_ids done
33 out of 989 srch_ids done
34 out of 989 srch_ids done
35 out of 989 srch_ids done
36 out of 9

380 out of 989 srch_ids done
381 out of 989 srch_ids done
382 out of 989 srch_ids done
383 out of 989 srch_ids done
384 out of 989 srch_ids done
385 out of 989 srch_ids done
386 out of 989 srch_ids done
387 out of 989 srch_ids done
388 out of 989 srch_ids done
389 out of 989 srch_ids done
390 out of 989 srch_ids done
391 out of 989 srch_ids done
392 out of 989 srch_ids done
393 out of 989 srch_ids done
394 out of 989 srch_ids done
395 out of 989 srch_ids done
396 out of 989 srch_ids done
397 out of 989 srch_ids done
398 out of 989 srch_ids done
399 out of 989 srch_ids done
400 out of 989 srch_ids done
401 out of 989 srch_ids done
402 out of 989 srch_ids done
403 out of 989 srch_ids done
404 out of 989 srch_ids done
405 out of 989 srch_ids done
406 out of 989 srch_ids done
407 out of 989 srch_ids done
408 out of 989 srch_ids done
409 out of 989 srch_ids done
410 out of 989 srch_ids done
411 out of 989 srch_ids done
412 out of 989 srch_ids done
413 out of 989 srch_ids done
414 out of 989

754 out of 989 srch_ids done
755 out of 989 srch_ids done
756 out of 989 srch_ids done
757 out of 989 srch_ids done
758 out of 989 srch_ids done
759 out of 989 srch_ids done
760 out of 989 srch_ids done
761 out of 989 srch_ids done
762 out of 989 srch_ids done
763 out of 989 srch_ids done
764 out of 989 srch_ids done
765 out of 989 srch_ids done
766 out of 989 srch_ids done
767 out of 989 srch_ids done
768 out of 989 srch_ids done
769 out of 989 srch_ids done
770 out of 989 srch_ids done
771 out of 989 srch_ids done
772 out of 989 srch_ids done
773 out of 989 srch_ids done
774 out of 989 srch_ids done
775 out of 989 srch_ids done
776 out of 989 srch_ids done
777 out of 989 srch_ids done
778 out of 989 srch_ids done
779 out of 989 srch_ids done
780 out of 989 srch_ids done
781 out of 989 srch_ids done
782 out of 989 srch_ids done
783 out of 989 srch_ids done
784 out of 989 srch_ids done
785 out of 989 srch_ids done
786 out of 989 srch_ids done
787 out of 989 srch_ids done
788 out of 989

In [9]:
gb_scores_3cat

[0.4745339045996835,
 0.49176383473906954,
 0.5004174763423332,
 0.4999497159782564,
 0.48680191683052404,
 0.4871745640157661,
 0.49674917987913697,
 0.5002552551274757,
 0.47544808577549125,
 0.48498416370131175]

In [ ]:
features2 = ['site_id', 'visitor_location_country_id', 'prop_country_id', 'prop_review_score', 'prop_brand_bool', 'prop_location_score1', 'prop_location_score2', 'promotion_flag', 'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool', 'srch_query_affinity_score', 'orig_destination_distance', 'random_bool', 'star_diff_bool', 'price_diff_bool', 'star_diff_filled', 'price_diff_filled', 'willingness_to_pay', 'month']
gb_scores2, gb_feature_importances2 = fit_predict_evaluate(train_sample, GradientBoostingClassifier(), features2, 'booking_bool')

fit model
predict


c:\users\fiona\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\fiona\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


1 out of 1060 srch_ids done
2 out of 1060 srch_ids done
3 out of 1060 srch_ids done
4 out of 1060 srch_ids done
5 out of 1060 srch_ids done
6 out of 1060 srch_ids done
7 out of 1060 srch_ids done
8 out of 1060 srch_ids done
9 out of 1060 srch_ids done
10 out of 1060 srch_ids done
11 out of 1060 srch_ids done
12 out of 1060 srch_ids done
13 out of 1060 srch_ids done
14 out of 1060 srch_ids done
15 out of 1060 srch_ids done
16 out of 1060 srch_ids done
17 out of 1060 srch_ids done
18 out of 1060 srch_ids done
19 out of 1060 srch_ids done
20 out of 1060 srch_ids done
21 out of 1060 srch_ids done
22 out of 1060 srch_ids done
23 out of 1060 srch_ids done
24 out of 1060 srch_ids done
25 out of 1060 srch_ids done
26 out of 1060 srch_ids done
27 out of 1060 srch_ids done
28 out of 1060 srch_ids done
29 out of 1060 srch_ids done
30 out of 1060 srch_ids done
31 out of 1060 srch_ids done
32 out of 1060 srch_ids done
33 out of 1060 srch_ids done
34 out of 1060 srch_ids done
35 out of 1060 srch_ids

298 out of 1060 srch_ids done
299 out of 1060 srch_ids done
300 out of 1060 srch_ids done
301 out of 1060 srch_ids done
302 out of 1060 srch_ids done
303 out of 1060 srch_ids done
304 out of 1060 srch_ids done
305 out of 1060 srch_ids done
306 out of 1060 srch_ids done
307 out of 1060 srch_ids done
308 out of 1060 srch_ids done
309 out of 1060 srch_ids done
310 out of 1060 srch_ids done
311 out of 1060 srch_ids done
312 out of 1060 srch_ids done
313 out of 1060 srch_ids done
314 out of 1060 srch_ids done
315 out of 1060 srch_ids done
316 out of 1060 srch_ids done
317 out of 1060 srch_ids done
318 out of 1060 srch_ids done
319 out of 1060 srch_ids done
320 out of 1060 srch_ids done
321 out of 1060 srch_ids done
322 out of 1060 srch_ids done
323 out of 1060 srch_ids done
324 out of 1060 srch_ids done
325 out of 1060 srch_ids done
326 out of 1060 srch_ids done
327 out of 1060 srch_ids done
328 out of 1060 srch_ids done
329 out of 1060 srch_ids done
330 out of 1060 srch_ids done
331 out of

619 out of 1060 srch_ids done
620 out of 1060 srch_ids done
621 out of 1060 srch_ids done
622 out of 1060 srch_ids done
623 out of 1060 srch_ids done
624 out of 1060 srch_ids done
625 out of 1060 srch_ids done
626 out of 1060 srch_ids done
627 out of 1060 srch_ids done
628 out of 1060 srch_ids done
629 out of 1060 srch_ids done
630 out of 1060 srch_ids done
631 out of 1060 srch_ids done
632 out of 1060 srch_ids done
633 out of 1060 srch_ids done
634 out of 1060 srch_ids done
635 out of 1060 srch_ids done
636 out of 1060 srch_ids done
637 out of 1060 srch_ids done
638 out of 1060 srch_ids done
639 out of 1060 srch_ids done
640 out of 1060 srch_ids done
641 out of 1060 srch_ids done
642 out of 1060 srch_ids done
643 out of 1060 srch_ids done
644 out of 1060 srch_ids done
645 out of 1060 srch_ids done
646 out of 1060 srch_ids done
647 out of 1060 srch_ids done
648 out of 1060 srch_ids done
649 out of 1060 srch_ids done
650 out of 1060 srch_ids done
651 out of 1060 srch_ids done
652 out of

...

589 out of 999 srch_ids done
590 out of 999 srch_ids done
591 out of 999 srch_ids done
592 out of 999 srch_ids done
593 out of 999 srch_ids done
594 out of 999 srch_ids done
595 out of 999 srch_ids done
596 out of 999 srch_ids done
597 out of 999 srch_ids done
598 out of 999 srch_ids done
599 out of 999 srch_ids done
600 out of 999 srch_ids done
601 out of 999 srch_ids done
602 out of 999 srch_ids done
603 out of 999 srch_ids done
604 out of 999 srch_ids done
605 out of 999 srch_ids done
606 out of 999 srch_ids done
607 out of 999 srch_ids done
608 out of 999 srch_ids done
609 out of 999 srch_ids done
610 out of 999 srch_ids done
611 out of 999 srch_ids done
612 out of 999 srch_ids done
613 out of 999 srch_ids done
614 out of 999 srch_ids done
615 out of 999 srch_ids done
616 out of 999 srch_ids done
617 out of 999 srch_ids done
618 out of 999 srch_ids done
619 out of 999 srch_ids done
620 out of 999 srch_ids done
621 out of 999 srch_ids done
622 out of 999 srch_ids done
623 out of 999

875 out of 999 srch_ids done
876 out of 999 srch_ids done
877 out of 999 srch_ids done
878 out of 999 srch_ids done
879 out of 999 srch_ids done
880 out of 999 srch_ids done
881 out of 999 srch_ids done
882 out of 999 srch_ids done
883 out of 999 srch_ids done
884 out of 999 srch_ids done
885 out of 999 srch_ids done
886 out of 999 srch_ids done
887 out of 999 srch_ids done
888 out of 999 srch_ids done
889 out of 999 srch_ids done
890 out of 999 srch_ids done
891 out of 999 srch_ids done
892 out of 999 srch_ids done
893 out of 999 srch_ids done
894 out of 999 srch_ids done
895 out of 999 srch_ids done
896 out of 999 srch_ids done
897 out of 999 srch_ids done
898 out of 999 srch_ids done
899 out of 999 srch_ids done
900 out of 999 srch_ids done
901 out of 999 srch_ids done
902 out of 999 srch_ids done
903 out of 999 srch_ids done
904 out of 999 srch_ids done
905 out of 999 srch_ids done
906 out of 999 srch_ids done
907 out of 999 srch_ids done
908 out of 999 srch_ids done
909 out of 999

159 out of 1020 srch_ids done
160 out of 1020 srch_ids done
161 out of 1020 srch_ids done
162 out of 1020 srch_ids done
163 out of 1020 srch_ids done
164 out of 1020 srch_ids done
165 out of 1020 srch_ids done
166 out of 1020 srch_ids done
167 out of 1020 srch_ids done
168 out of 1020 srch_ids done
169 out of 1020 srch_ids done
170 out of 1020 srch_ids done
171 out of 1020 srch_ids done
172 out of 1020 srch_ids done
173 out of 1020 srch_ids done
174 out of 1020 srch_ids done
175 out of 1020 srch_ids done
176 out of 1020 srch_ids done
177 out of 1020 srch_ids done
178 out of 1020 srch_ids done
179 out of 1020 srch_ids done
180 out of 1020 srch_ids done
181 out of 1020 srch_ids done
182 out of 1020 srch_ids done
183 out of 1020 srch_ids done
184 out of 1020 srch_ids done
185 out of 1020 srch_ids done
186 out of 1020 srch_ids done
187 out of 1020 srch_ids done
188 out of 1020 srch_ids done
189 out of 1020 srch_ids done
190 out of 1020 srch_ids done
191 out of 1020 srch_ids done
192 out of

476 out of 1020 srch_ids done
477 out of 1020 srch_ids done
478 out of 1020 srch_ids done
479 out of 1020 srch_ids done
480 out of 1020 srch_ids done
481 out of 1020 srch_ids done
482 out of 1020 srch_ids done
483 out of 1020 srch_ids done
484 out of 1020 srch_ids done
485 out of 1020 srch_ids done
486 out of 1020 srch_ids done
487 out of 1020 srch_ids done
488 out of 1020 srch_ids done
489 out of 1020 srch_ids done
490 out of 1020 srch_ids done
491 out of 1020 srch_ids done
492 out of 1020 srch_ids done
493 out of 1020 srch_ids done
494 out of 1020 srch_ids done
495 out of 1020 srch_ids done
496 out of 1020 srch_ids done
497 out of 1020 srch_ids done
498 out of 1020 srch_ids done
499 out of 1020 srch_ids done
500 out of 1020 srch_ids done
501 out of 1020 srch_ids done
502 out of 1020 srch_ids done
503 out of 1020 srch_ids done
504 out of 1020 srch_ids done
505 out of 1020 srch_ids done
506 out of 1020 srch_ids done
507 out of 1020 srch_ids done
508 out of 1020 srch_ids done
509 out of

782 out of 1020 srch_ids done
783 out of 1020 srch_ids done
784 out of 1020 srch_ids done
785 out of 1020 srch_ids done
786 out of 1020 srch_ids done
787 out of 1020 srch_ids done
788 out of 1020 srch_ids done
789 out of 1020 srch_ids done
790 out of 1020 srch_ids done
791 out of 1020 srch_ids done
792 out of 1020 srch_ids done
793 out of 1020 srch_ids done
794 out of 1020 srch_ids done
795 out of 1020 srch_ids done
796 out of 1020 srch_ids done
797 out of 1020 srch_ids done
798 out of 1020 srch_ids done
799 out of 1020 srch_ids done
800 out of 1020 srch_ids done
801 out of 1020 srch_ids done
802 out of 1020 srch_ids done
803 out of 1020 srch_ids done
804 out of 1020 srch_ids done
805 out of 1020 srch_ids done
806 out of 1020 srch_ids done
807 out of 1020 srch_ids done
808 out of 1020 srch_ids done
809 out of 1020 srch_ids done
810 out of 1020 srch_ids done
811 out of 1020 srch_ids done
812 out of 1020 srch_ids done
813 out of 1020 srch_ids done
814 out of 1020 srch_ids done
815 out of

In [11]:
gb_scores2

[0.42895065971984886,
 0.4318961068131547,
 0.43606565432395644,
 0.43402261468354436,
 0.4220017295576636,
 0.4531147960443147,
 0.44658877622196114,
 0.42979855004462586,
 0.4370563943597479,
 0.4275688095372029]